In [3]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/santandar-processed-data/fdata.csv
/kaggle/input/vip-preprocessed/VIP data.csv


In [4]:
VIP_data = pd.read_csv("/kaggle/input/vip-preprocessed/VIP data.csv")
VIP_data .head()

,data_date,employee_index,country_residence,gender,registration_date,new_customer_index,seniority,primary_customer,customer_type_beginning_month,customer_relation_type_beginning_month,...,prod_pensions1,prod_loans,prod_taxes,prod_credit_card,prod_securities,prod_home_account,prod_payroll,prod_pensions2,prod_direct_debit,age_group
0,2015-01-28,3,0,0,2012-08-10,0,35.0,1.0,1,A,...,0,0,0,0,0,0,0.0,1.0,1,4
1,2015-01-28,3,0,1,2012-08-14,0,35.0,1.0,1,A,...,0,0,0,0,0,0,0.0,0.0,0,2
2,2015-01-28,3,0,0,2012-08-16,0,35.0,1.0,1,A,...,0,0,0,0,0,0,0.0,0.0,0,3
3,2015-01-28,3,0,1,2012-08-16,0,35.0,1.0,1,A,...,0,0,0,0,0,0,0.0,0.0,1,3
4,2015-01-28,3,0,1,2012-08-13,0,35.0,1.0,1,A,...,0,0,0,1,0,0,0.0,0.0,1,2


In [5]:
from sklearn.preprocessing import LabelEncoder
date_columns = ['data_date', 'registration_date']
for col in date_columns:
    print(col)
    VIP_data[col] = pd.to_datetime(VIP_data[col])

categorical_columns = ['customer_relation_type_beginning_month', 'residence_index', 'foreigner_index', 'channel_used', 'province_name', 'segmentation']
label_encoders = {}
for col in categorical_columns:
    print(col)
    label_encoders[col] = LabelEncoder()
    VIP_data[col] = label_encoders[col].fit_transform(VIP_data[col].astype(str))


data_date
registration_date
customer_relation_type_beginning_month
residence_index
foreigner_index
channel_used
province_name
segmentation


In [6]:
for col in date_columns:
    print(col)
    VIP_data[col] = (VIP_data[col] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

data_date
registration_date


In [7]:
VIP_data_cleaned = VIP_data.dropna()

In [8]:
VIP_data_cleaned .isnull().sum()

data_date                                 0
employee_index                            0
country_residence                         0
gender                                    0
registration_date                         0
new_customer_index                        0
seniority                                 0
primary_customer                          0
customer_type_beginning_month             0
customer_relation_type_beginning_month    0
residence_index                           0
foreigner_index                           0
channel_used                              0
province_code                             0
province_name                             0
activity_index                            0
gross_income_household                    0
segmentation                              0
prod_savings_account                      0
prod_guarantees                           0
prod_current_accounts                     0
prod_derivative_account                   0
prod_payroll_account            

In [9]:
VIP_data_cleaned.iloc[[0]]

,data_date,employee_index,country_residence,gender,registration_date,new_customer_index,seniority,primary_customer,customer_type_beginning_month,customer_relation_type_beginning_month,...,prod_pensions1,prod_loans,prod_taxes,prod_credit_card,prod_securities,prod_home_account,prod_payroll,prod_pensions2,prod_direct_debit,age_group
0,1422403200,3,0,0,1344556800,0,35.0,1.0,1,0,...,0,0,0,0,0,0,0.0,1.0,1,4


In [11]:
VIP_data_cleaned.shape

(562120, 43)

# Synthetic Data Generation


# GMM Variate

In [ ]:
# !pip install copulas

# import pandas as pd
# from copulas.multivariate import GaussianMultivariate
# from copulas.visualization import scatter_2d
# import warnings
# from tqdm import tqdm

# warnings.filterwarnings("ignore")

# # Restrict the number of rows to 10,000
# vip_data1 = vip_data.head(100)

# # Fit copula model
# copula = GaussianMultivariate()
# copula.fit(vip_data1)

# # Generate synthetic data
# synthetic_data = []
# total_samples = len(vip_data1)
# with tqdm(total=total_samples, desc="Generating Synthetic Data") as pbar:
#     for _ in range(total_samples):
#         sample = copula.sample(1)
#         synthetic_data.append(sample)
#         pbar.update(1)

# # Concatenate the synthetic samples
# synthetic_data = pd.concat(synthetic_data, ignore_index=True)

In [12]:
vip_data = VIP_data_cleaned

In [13]:
import numpy as np
import matplotlib.pyplot as plt

vip_data1 = vip_data.head(100)
n_components = range(1, 20)
models = [GaussianMixture(n, random_state=42).fit(vip_data1) for n in n_components]


aic = [model.aic(vip_data1) for model in models]
bic = [model.bic(vip_data1) for model in models]


plt.plot(n_components, aic, label='AIC')
plt.plot(n_components, bic, label='BIC')
plt.xlabel('Number of Components')
plt.ylabel('Criterion Value')
plt.legend()
plt.show()


NameError: name 'GaussianMixture' is not defined

In [ ]:
import pandas as pd
from sklearn.mixture import GaussianMixture
from tqdm import tqdm
import warnings

warnings.filterwarnings("ignore")

vip_data1 = vip_data.head(10)

gmm = GaussianMixture(n_components=3) 
gmm.fit(vip_data1)

synthetic_data = []
total_samples = len(vip_data1)
with tqdm(total=total_samples, desc="Generating Synthetic Data") as pbar:
    for _ in range(total_samples):
        sample = gmm.sample(1)
        synthetic_data.append(sample[0].flatten())  # Flatten the 3D array to 1D
        pbar.update(1)

synthetic_data = pd.DataFrame(synthetic_data, columns=vip_data1.columns)


In [ ]:
synthetic_data.head()

In [ ]:
# for col in VIP_data_cleaned.columns.tolist():
#     print(f"Unique values in {col}: {VIP_data_cleaned[col].unique()}")

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

def replace_with_closest_unique_value(value, column_unique_values):
    return min(column_unique_values, key=lambda x: abs(x - value))

columns_to_process = synthetic_data.columns.difference(['gross_income_household', 'data_date', 'registration_date'])

for index, row in tqdm(synthetic_data.iterrows(), total=len(synthetic_data)):
    for col in columns_to_process:
        unique_values = VIP_data_cleaned[col].unique()
        if len(unique_values) > 1:  # Skip if there's only one unique value
            synthetic_data.at[index, col] = replace_with_closest_unique_value(row[col], unique_values)

In [ ]:
synthetic_data.head()

# KDE

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KernelDensity



# List of columns to use for KDE (excluding any non-numeric columns)
numeric_columns = VIP_data_cleaned.select_dtypes(include=np.number).columns
print(numeric_columns)
# Fit KDE for each numeric column
kde_models = {}
for col in numeric_columns:
    kde = KernelDensity(bandwidth=0.1)  # You may need to adjust the bandwidth
    kde.fit(VIP_data_cleaned[[col]])
    kde_models[col] = kde

# Number of synthetic samples to generate (adjust as needed)
num_samples = 1000

# Generate synthetic data
synthetic_data = pd.DataFrame()
for col, kde in kde_models.items():
    synthetic_samples = kde.sample(num_samples)
    synthetic_data[col] = synthetic_samples.flatten()


In [ ]:
synthetic_data.head()

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

def replace_with_closest_unique_value(value, column_unique_values):
    return min(column_unique_values, key=lambda x: abs(x - value))

columns_to_process = synthetic_data.columns.difference(['gross_income_household', 'data_date', 'registration_date'])

for index, row in tqdm(synthetic_data.iterrows(), total=len(synthetic_data)):
    for col in columns_to_process:
        unique_values = VIP_data_cleaned[col].unique()
        if len(unique_values) > 1:  # Skip if there's only one unique value
            synthetic_data.at[index, col] = replace_with_closest_unique_value(row[col], unique_values)

In [ ]:
synthetic_data.head()